In [61]:
import sqlite3
from pymystem3 import Mystem
m = Mystem()
con = sqlite3.connect('FDB.db')
cur = con.cursor()

In [3]:
import pandas as pd
import re

# 0 Data preparation

Tags

In [3]:
with open('./raw_data/tags.csv','r') as f:
    tags = f.readlines()
    result = {t.split()[0]:t.split()[1] for t in tags}
print (result)

{'A': 'pos', 'ADV': 'pos', 'ADVPRO': 'pos', 'ANUM': 'pos', 'APRO': 'pos', 'COM': 'pos', 'CONJ': 'pos', 'INTJ': 'pos', 'NUM': 'pos', 'PART': 'pos', 'PR': 'pos', 'S': 'pos', 'SPRO': 'pos', 'V': 'pos', 'praes': 'tense', 'inpraes': 'tense', 'praet': 'tense', 'nom': 'case', 'gen': 'case', 'dat': 'case', 'acc': 'case', 'ins': 'case', 'abl': 'case', 'part': 'case', 'loc': 'case', 'voc': 'case', 'sg': 'num', 'pl': 'num', 'ger': 'vform', 'inf': 'vform', 'partcp': 'vform', 'indic': 'vform', 'imper': 'vform', 'brev': 'aform', 'plen': 'aform', 'poss': 'aform', 'supr': 'degree', 'comp': 'degree', '1p': 'pers', '2p': 'pers', '3p': 'pers', 'm': 'gender', 'f': 'gender', 'n': 'gender', 'ipf': 'asp', 'pf': 'asp', 'act': 'voice', 'pass': 'voice', 'tran': 'trans', 'intr': 'trans', 'parenth': 'type', 'praed': 'type', 'abbr': 'type', 'mf': 'type', 'awkw': 'form', 'dist': 'form', 'geo': 'sem', 'persn': 'sem', 'patrn': 'sem', 'famn': 'sem', 'obsc': 'style', 'inform': 'style', 'rare': 'style', 'obsol': 'style'

# 1 block

**Import informants**

In [123]:
db_1 = sqlite3.connect('./raw_data/ABM.sqlite')
db_1_cur = db_1.cursor()

In [124]:
db_1_cur.execute('SELECT * FROM tblInformators')
inf = db_1_cur.fetchall()

In [127]:
cur.executemany('INSERT INTO Informators (code, name, city, bio, old_id) VALUES (?,?,?,?,?)', inf)

In [128]:
con.commit()

**Import texts to database**

load data

In [102]:
db_1_cur.execute('SELECT * FROM tblCards')
texts = db_1_cur.fetchall()

In [29]:
for key, value in enumerate(texts[0]):
    print (key, value)

0 [Шёл разговор о пьющей женщине из Карпиловки, Г назвал её «Ма\ндыка».] [Ма\ндыка?] [Г:] Да. [Что это значит?] [Г, смеясь:] Ну, манды\ка — э\то… [А одновременно:] Коро\че её ёб… [Г:] Все. [А:] Всё село\. [Г:] Да, коро\че… [А:] И… и… и из на\шего, и коро\че там… [Г:] Все подря\д. [А:] Да. [А и Г одновременно:] Кому\ не лень. [А:] Буты\лку на стол, и… [Она пьёт, да?] [Г:] Ну, она\ по-чё\рному пьёт.
1 
2 
3 
4 XXIa
5 5 б
6 Брянская обл., Злынковский р-н, Спиридонова Буда
7 2017
8 А
9 Г
10 
11 
12 МЭГ
13 НВП
14 СЕ
15 ФЕЕ
16 1
17 
18 
19 
20 


In [116]:
texts2 = []

In [117]:
for row in texts:
    new_row = (
        'АБМ',
        row[16],
        row[6],
        row[0],
        row[7],
        restore_label(row[5]),
        row[4],
        ', '.join(i for i in row[8:12] if i != ''),
        ', '.join(i for i in row[12:16] if i != '') 
    )
    texts2.append(new_row)

In [121]:
cur.executemany('INSERT INTO Texts (leader, old_id, city, raw_text, year, question, list, informants, collectors)\
VALUES (?,?,?,?,?,?,?,?,?)', texts2)

In [122]:
con.commit()

2. change question labels

In [110]:
def restore_label(label):
    label = label.split(',')
    result = []
    #print(label)
    for i in label:
        q = re.findall('^([0-9]*)[^0-9a-zа-я]*?([ a-zа-я\.]*?)$', i.strip())[0]
        #print (q)
        result.append((q[0].strip(), q[1].strip()))
    for key, value in enumerate(result):
        if value[0] == '':
            result[key] = (result[key-1][0].strip(), value[1].strip())
    #print (result)
    return ', '.join(''.join(i) for i in result)

In [115]:
restore_label('15 а, 19 а, в, 30 а, 14 а, 25 а, б, 16 а')

'15а, 19а, 19в, 30а, 14а, 25а, 25б, 16а'

In [68]:
re.findall('^([0-9]*)[^0-9a-zа-я]*?([a-zа-я]*?)$',label[1].strip())

[('10', 'б')]

In [ ]:
data['Вопрос'] = restore_label(data['Вопрос'])

3. normalize text

In [215]:
a = 'у% Я́ цветоу%ок не брал и цветоу%'
len ([i['text'] for i in m.analyze(a)]) == len(m.analyze(a))

True

In [216]:
def norm_v(text):
    k = [i['text'] for i in m.analyze(text)]
    r = ['empty']
    for i in k:
        if '%' not in i:
            if r[-1][-1] != '%':
                r += [i]
            else:
                r[-1]+=i
        else:
            r[-1]+='%'
            if i != '%':
                r += [i.replace('%','')] 
    return r[1:]

In [218]:
norm_v(a)

['у%',
 ' ',
 'Я́',
 ' ',
 'цветоу%ок',
 ' ',
 'не',
 ' ',
 'брал',
 ' ',
 'и',
 ' ',
 'цветоу%',
 '\n']

In [85]:
''.join(norm_v(a)).replace('у%','в')

'в Я́ цветовок не брал и цветов\n'

In [ ]:
def 

In [67]:
accents = {'А́ ':'А', 'Е́ ':'Е','И́ ':'И','О́ ':'О','У́ ':'У','Ы́ ':'Ы','Э́ ':'Э','Ю́ ':'Ю','Я́':'Я',
           'а́':'а','е́':'е','и́':'и','о́':'о', 'у́':'у', 'ы́':'ы', 'э́':'э', 'ю́':'ю', 'я́':'я'}

In [68]:
get_accents = {item[1]+'\\': item[0] for item in accents.items()}

In [168]:
a = '[Вы вчера рассказывали историю с цветами, что это было?] [А:] Ну, э\то как бы э… не зна\ю, и\ли приме\та и\ли… ну, что-то… вот когда\ моего\ дя\дю хорони\ли э… то\же, ну, кода\ хорони\ли его\, хоте\ли то\же цветы\ положи\ть и\ли что-то хоте\ли положи\ть туда\, вро\де, цветы\, она\ стро\го запрети\ла, ну там, же\нщина, кото\рая всё по э\тим, по похорона\м от, она\ сказа\ла: «Стро\го нельзя\». Ну, ти\па, ско\ка положи\ли тогда\, ну, в про\шлый раз там что-то, в моги\лу цвето\у%, сто… че\рез сто\ка дней ро\дственник у\мер и\ли чт… ну, коро\че, что-то тако\е, и\ли когда\ цветы\ завя\ли, ро\дственник у\мер, что-то… [Соб.: Ты что-то рассказывал, что она букет кинула в гроб…] Ну да, э\то вон, Гри\ша, да. [Соб.: А, Гриша.] [Г:] Я расска\зывал от что, ну, они\ сказа\ли то, что кода\ она\ ски\нула, ско\ка, то\ ли де\вять цвето\в, ну, в моги\лу получа\ется кода\... все песо\к кида\ли, а она\ цветы\ ски\нула. И по\сле э\того у нас, ну, начало\сь, вот э\ти де\вять пацано\в у\мерло. И\менно вот из-за э\того. Говоря\т то, что и\менно из-за э\того, то что…'

In [88]:
s = set(re.findall('.\\\\', a))
s

{'И\\', 'а\\', 'е\\', 'и\\', 'о\\', 'у\\', 'ы\\', 'э\\', 'я\\'}

In [108]:
for i in s:
    a = a.replace(i, get_accents[i])

In [109]:
a

'[Вы вчера рассказывали историю с цветами, что это было?] [А:] Ну, э́то как бы э… не зна́ю, и́ли приме́та и́ли… ну, что-то… вот когда́ моего́ дя́дю хорони́ли э… то́же, ну, кода́ хорони́ли его́, хоте́ли то́же цветы́ положи́ть и́ли что-то хоте́ли положи́ть туда́, вро́де, цветы́, она́ стро́го запрети́ла, ну там, же́нщина, кото́рая всё по э́тим, по похорона́м от, она́ сказа́ла: «Стро́го нельзя́». Ну, ти́па, ско́ка положи́ли тогда́, ну, в про́шлый раз там что-то, в моги́лу цвето́у%, сто… че́рез сто́ка дней ро́дственник у́мер и́ли чт… ну, коро́че, что-то тако́е, и́ли когда́ цветы́ завя́ли, ро́дственник у́мер, что-то… [Соб.: Ты что-то рассказывал, что она букет кинула в гроб…] Ну да, э́то вон, Гри́ша, да. [Соб.: А, Гриша.] [Г:] Я расска́зывал от что, ну, они́ сказа́ли то, что кода́ она́ ски́нула, ско́ка, то́ ли де́вять цвето́в, ну, в моги́лу получа́ется кода́... все песо́к кида́ли, а она́ цветы́ ски́нула. И по́сле э́того у нас, ну, начало́сь, вот э́ти де́вять пацано́в у́мерло. И́ менно вот из

In [113]:
a = re.sub('\[','\n[',a)

a = re.sub('[^:]\]',']\n',a)

a = re.sub('[ ]*?\n','\n',a)

a = re.sub('\n[ \n]*','\n',a).strip()

a = a.split('\n')

In [151]:
for i in a:
    r = re.findall('\[.*?\]', i)
    if len(r) > 0:
        if re.match('^\[[A-ZА-Я-0-9]{1,5}:\]$', r[0]):
            print (r[0])

[А:]
[Г:]


In [41]:
m.analyze(a)[22:25]

[{'analysis': [{'lex': 'ну', 'wt': 0.9991076439, 'gr': 'PART='}],
  'text': 'Ну'},
 {'text': ', '},
 {'analysis': [{'lex': 'это',
    'wt': 0.7809833731,
    'gr': 'SPRO,sg,n,inan=(acc|nom)'}],
  'text': 'э́то'}]

In [4]:
class OneSentence:
    def __init__(self, text, prefix):
        self.show_prefix = False
        self.prefix = prefix
        self.text = text
        
    def __str__(self):
        return '({}\t{}\t{})'.format(str(self.show_prefix), self.prefix, self.text)
    
    __repr__ = __str__
        
class Sentences:
    def __init__(self,text):
        self.sentences = self._assign_turns(text)
        #print (self.turns)
        
    def _split_turns(self, text):
        global get_accents
        tags = set(re.findall('.\\\\', text))
        for i in tags:
            text = text.replace(i, get_accents[i])
        text = re.sub('\[','\n[',text)
        text = re.sub('[^:]\]',']\n',text)
        text = re.sub('[ ]*?\n','\n',text)
        text = re.sub('\n[ \n]*','\n',text).strip()
        text = text.split('\n')
        return text
    
    def _assign_turns(self,text):
        text = self._split_turns(text)
        prefixes = []
        result = []
        for i in text:
            r = re.findall('\[.*?\]', i)
            if len(r) > 0:
                if re.match('^\[[A-ZА-Я-0-9]{1,5}:\]$', r[0]):
                    prefixes.append(r[0])
                    sentences = sent_tokenize(i.replace(r[0], '').strip())
                    result.append(OneSentence(sentences[0], prefixes[-1]))
                    result[-1].show_prefix = True
                    for x in sentences[1:]:
                        result.append(OneSentence(x, prefixes[-1]))
                else:
                    result.append(OneSentence(r[0], ''))
            else:
                sentences = sent_tokenize(i.strip())
                result.append(OneSentence(sentences[0], prefixes[-1]))
                result[-1].show_prefix = True
                for x in sentences[1:]:
                    result.append(OneSentence(x, prefixes[-1]))
        return result
    
    def __str__(self):
        return '\n\n'.join([str(i) for i in self.sentences])
    
    __repr__ = __str__

In [5]:
class TaggedSentences:
    
    def __init__(self, sentences, informators, city):
        self.sentences = sentences
        self.meta = self._get_inform(informators, city)
        
    def _norm_v(self, text):
        global accents
        k = [i['text'] for i in m.analyze(text)]
        r = ['empty']
        for i in k:
            if '%' not in i:
                if r[-1][-1] != '%':
                    r += [i]
                else:
                    r[-1]+=i
            else:
                r[-1]+='%'
                if i != '%':
                    r += [i.replace('%','')] 
        norm = ''.join(r[1:]).replace('у%','в').replace('У%','В')
        for i in accents:
            norm = norm.replace(i, accents[i])
        return r[1:], m.analyze(norm)
    
    def _get_inform(self, informators, city):
        global cur
        informators = [i.strip() for i in informators.split(',')]
        meta = {}
        for i in informators:
            

SyntaxError: invalid syntax (<ipython-input-5-f19f4878dc41>, line 25)

In [10]:
def _get_inform(informators, city):
        global cur
        informators = [i.strip() for i in informators.split(',')]
        meta = {}
        city = city.split(',')[-1].strip()
        for i in informators:
            cur.execute('SELECT id, code, birthplace, birthyear, gender FROM Informators WHERE\
            code=\"{}\" AND city=\"{}\"'.format(i, city))
            print (cur.fetchall())

In [15]:
cur.execute('SELECT informants, city from Texts')
data = cur.fetchall()

In [ ]:
for i in data:
    _get_inform(*i)

In [65]:
def norm_v(text):
        global accents
        k = [i['text'] for i in m.analyze(text)]
        r = ['empty']
        for i in k:
            if '%' not in i:
                if r[-1][-1] != '%':
                    r += [i]
                else:
                    r[-1]+=i
            else:
                r[-1]+='%'
                if i != '%':
                    r += [i.replace('%','')] 
        norm = ''.join(r[1:]).replace('у%','в').replace('У%','В')
        for i in accents:
            norm = norm.replace(i, accents[i])
        return r[1:], m.analyze(norm)

In [229]:
a = 'У% Я́ цветоу%ок не брал и цвето́у%'
norm_v(a)

(['У%',
  ' ',
  'Я́',
  ' ',
  'цветоу%ок',
  ' ',
  'не',
  ' ',
  'брал',
  ' ',
  'и',
  ' ',
  'цвето́у%',
  '\n'],
 [{'analysis': [{'lex': 'в',
     'wt': 8.212235587e-06,
     'gr': 'S,abbr=(abl,pl|abl,sg|acc,pl|acc,sg|dat,pl|dat,sg|gen,pl|gen,sg|ins,pl|ins,sg|nom,pl|nom,sg)'}],
   'text': 'В'},
  {'text': ' '},
  {'analysis': [{'lex': 'я',
     'wt': 2.837188476e-05,
     'gr': 'S,abbr=(abl,pl|abl,sg|acc,pl|acc,sg|dat,pl|dat,sg|gen,pl|gen,sg|ins,pl|ins,sg|nom,pl|nom,sg)'}],
   'text': 'Я'},
  {'text': ' '},
  {'analysis': [{'lex': 'цветовка',
     'wt': 1,
     'qual': 'bastard',
     'gr': 'S,geo,f,inan=gen,pl'}],
   'text': 'цветовок'},
  {'text': ' '},
  {'analysis': [{'lex': 'не', 'wt': 1, 'gr': 'PART='}], 'text': 'не'},
  {'text': ' '},
  {'analysis': [{'lex': 'брать',
     'wt': 1,
     'gr': 'V,ipf,tran=praet,sg,indic,m'}],
   'text': 'брал'},
  {'text': ' '},
  {'analysis': [{'lex': 'и', 'wt': 0.9999770357, 'gr': 'CONJ='}], 'text': 'и'},
  {'text': ' '},
  {'analysis': 

In [181]:
from nltk.tokenize import sent_tokenize

In [210]:
A = Sentences(a)
A

(False		[Вы вчера рассказывали историю с цветами, что это было])

(True	[А:]	Ну, э́то как бы э… не зна́ю, и́ли приме́та и́ли… ну, что-то… вот когда́ моего́ дя́дю хорони́ли э… то́же, ну, кода́ хорони́ли его́, хоте́ли то́же цветы́ положи́ть и́ли что-то хоте́ли положи́ть туда́, вро́де, цветы́, она́ стро́го запрети́ла, ну там, же́нщина, кото́рая всё по э́тим, по похорона́м от, она́ сказа́ла: «Стро́го нельзя́».)

(False	[А:]	Ну, ти́па, ско́ка положи́ли тогда́, ну, в про́шлый раз там что-то, в моги́лу цвето́у%, сто… че́рез сто́ка дней ро́дственник у́мер и́ли чт… ну, коро́че, что-то тако́е, и́ли когда́ цветы́ завя́ли, ро́дственник у́мер, что-то…)

(False		[Соб.: Ты что-то рассказывал, что она букет кинула в гроб])

(True	[А:]	Ну да, э́то вон, Гри́ша, да.)

(False		[Соб.: А, Гриша])

(True	[Г:]	Я расска́зывал от что, ну, они́ сказа́ли то, что кода́ она́ ски́нула, ско́ка, то́ ли де́вять цвето́в, ну, в моги́лу получа́ется кода́... все песо́к кида́ли, а она́ цветы́ ски́нула.)

(False	[Г:]	И по́сл

In [213]:
text = A.sentences[2].text

In [ ]:
m.analyze

In [133]:
def no_stress(text):
    text = text.replace('\\', '')
    return text

def split_sentences(text):
    text = text.replace('[', '\n[')
    text = re.sub('\n{2,}','\n')
    text = text.split('\n')
    text = [sent_tokenize(chunk) for chunk in text]
    return text

In [ ]:
def get_meta(code):
    cur.execute('SELECT id, gender, birthyear FROM Informants WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return {'id':indiv[0], 'gender':indiv[1], 'year':indiv[2]}

def get_col_id(code):
    cur.execute('SELECT id FROM Collectors WHERE code=\"{}\"'.format(code))
    indiv = cur.fetchone()
    return indiv[0]

def joint_meta(informants, collectors):
    informants = informants.split(',')
    result = {'inf':{}, 'col':{}}
    for i in informants:
        result['inf'][i.strip()] = get_meta(code)
    for i in collectors:
        result['col'][i.strip()] = get_col_id(code)

In [ ]:
def join_text(text, sentence):
    text = text
    for word in words:
            if 'wf' not in word:
                continue
            word['off_start'] = len(text)
            if word['wtype'] == 'word':
                text += word['wf'] + ' '
                word['off_end'] = len(text) - 1
            else:
                if word['wf'].startswith(('(', '[', '{', '<', '“')):
                    text += word['wf']
                    word['off_end'] = len(text)
                elif word['wf'].startswith((')', ']', '}', '>', '.', ',', '?', '!', '”', '…')):
                    if text.endswith(' '):
                        word['off_start'] -= 1
                        text = text[:-1]
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
                else:
                    text += word['wf'] + ' '
                    word['off_end'] = len(text) - 1
        return text.strip()

In [ ]:
def ana(analysis):
    global tags
    result = {}
    for i in analysis:
        if i in tags:
            result[tags[i]] = i
    return result

In [ ]:
def tokens(parsed):
    result = []
    for i in parsed:
        if 'analysis' in i:
            r = {}
            r['wf'] = i['text']
            r['wtype'] = i['word']
            r['ana'] = []
            for var in i['analysis']:
                r['ana'].append(ana(var))
    return result

In [ ]:
def parse_sentence(sentence, meta):
    result = {'meta':{}, 'words':[]}
    if not re.match('^\[.*?\]$', sentence):
        sentence = sentence.split(']')
        sentence[0] = sentence[0]+']'
        parsed = tokens(m.analyze(sentence[1]))
    else:
        parsed = tokens(m.analyze(sentence))
    

# Block 2

Informators

In [191]:
df = pd.read_csv('./raw_data/Список файлов - Лист1.csv') 

In [192]:
df.columns

Index(['Имя файла', 'Жанр', 'Ссылка на текстовый файл', 'Ссылка на аудио файл',
       'Деревня', 'ФИО', 'Год рождения', 'Место рождения',
       'Место записи (адрес, координаты, etc.)', 'ФИ собирателей',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [193]:
df.head(3)

,Имя файла,Жанр,Ссылка на текстовый файл,Ссылка на аудио файл,Деревня,ФИО,Год рождения,Место рождения,"Место записи (адрес, координаты, etc.)",ФИ собирателей,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,Архив ВШЭ-ФЭ-69-14-2017-1,Сведения об игре на гармони,https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN
1,Архив ВШЭ-ФЭ-69-14-2017-2,Молодежные посиделки,https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN
2,Архив ВШЭ-ФЭ-69-14-2017-3,"Сведения о песнях, танцах, репертуаре на молод...",https://github.com/expokimry17/texts/blob/mast...,NaN,пос.Белый городок,Кочнев Виктор Алексеевич,1934,Калязинский район,"пос. Белый городок, ул. Заводская д. 5","Надежда Григорьева, Аня Копаева",NaN,NaN,NaN


In [ ]:
set(df['ФИ собирателей'].values)

In [211]:
def collectors(names):
    w = names.split(', ')
    w = [''.join(c for c in i if c.isupper()) for i in w]
    w = [i[-1]+i[:-1] if i != 'ХТ' else i for i in w ]
    return ', '.join(w)

In [212]:
for i in set(df['ФИ собирателей'].values):
    print (collectors(i))

ДВЕ, КМ, ХТ
ГН, КА
ХТ
КЛ, ДВ, ГН
КЮМ, ХТ
ТО, КМ, МТ
ДВ
КЮМ, ГН, КЛ, ДВ
ХТ
КЮМ, ДВ
ТО, МТ
КЮМ
КМ, ХТ


Informants

In [197]:
inf = df[['ФИО', 'Деревня','Год рождения', 'Место рождения']].drop_duplicates(['ФИО'])

In [51]:
for i in range(len(inf)):
    d = inf.iloc[i]
    if ',' not in d['ФИО']:
        s = 'INSERT INTO Informators (name, city, birthplace, birthyear) \
        VALUES (\"{}\",\"{}\",\"{}\",\"{}\")'.format(
        d['ФИО'], d['Деревня'], d['Место рождения'], d['Год рождения'])
        cur.execute(s)

In [52]:
con.commit()

Texts

In [11]:
import os

In [13]:
os.path.exists('./raw_data/texts/Архив ВШЭ-ФЭ-69-14-2017-1')

False

In [146]:
def get_text(name):
    name = './raw_data/texts/{}'.format(name)
    if not os.path.exists(name): name = name + '.txt'
    try: 
        with open (name, 'r', encoding='utf-8') as f:
            return [i for i in f.read().strip().split('\n') if i != '']
    except UnicodeDecodeError:
        with open (name, 'r', encoding='cp1251') as f:
            return [i for i in f.read().strip().split('\n') if i != '']

In [236]:
get_text('Архив ВШЭ-ФЭ-69-14-2017-9')

['С2: А что за игры?',
 'ВА: Ну игры, так вот.. Значит, девушка садится, берёт полотенце, вот значит, а парень, вот значит, вот так вот.. ну чуть.. под 90 градусов наклоняется, а девушка ему глаза вот так вот заты- зажмуривает полотенцем, да? кладёшь руку на пояницу, вот себе на спину, ну и по ней кто-то бьёт ладонью по ладони по его. А он должен угадать. Если он сразу угадал, значит, отделался, а если он не угадывает, ему так руку набьют.. ой-ой-ой. Вот такая вот.',
 'С1: А как эта игра называлась?',
 'ВА: А это назывался «Овёс».',
 'С2: Как?',
 'ВА: "Овёс". Игра в "Овёс".',
 'С1: А почему "Овёс"?',
 'ВА: А если.. вот когда.. хто долго не угадывает, дак «Ну, всё, – говрит, – вдоль овса пошёл, вдоль поля». Всё, да, ему руку так наколотят, что.. да, но вот так.',
 'С1: А ещё во что играли?',
 'ВА: Что сейчас?',
 'С1: Нет, тогда, на поседках, во что ещё играли? Игры какие.']

In [47]:
def normalize_view(text_array, informants:list):
    inf = {i:i for i in informants}
    copy = informants
    informants = {}
    informants.update({i[-2]:i for i in inf})
    informants.update({i[-2:]:i for i in inf})
    informants.update({i:i for i in inf})
    #if copy == ['ОРА']: informants = {'РА':'ОГА'}
    #print (informants)
    result = []
    for i in text_array:
        if re.match('^[CС0-9, -]{1,7}:.*?$', i):
            result.append('[{}]'.format(i))
            #print ('[{}]'.format(i))
        else:
            r = re.findall('^([А-Яа-яA-Za-z0-9-, ]{1,7}):(.*?$)',i)
            if r:
                    prefix, text = r[0]
                    prefix = ''.join(c for c in prefix if c.isupper() or c.isdigit() or c in ', ')
                    try:
                        result.append('[{}:]{}'.format(informants[prefix], text)) 
                    except KeyError:
                        result.append('[{}:]{}'.format(prefix, text)) 
            else:
                try:
                    result[-1] = result[-1] + ' ' + i
                except:
                    result.append(i)
    result = '\n'.join(result)
    result = result.replace('«','"').replace('»','"')
    return result

In [85]:
print(normalize_view(array, ['КВА']))

[С2: А как поседки проходили? Что делали?]
[КВА:] Поседки, раньше поседки.. каждая девушка, она должна справить в своём дому вечер. Пускали, молодёжь собирается, ну и на поседках всё там: и танцуют, и пляшут, и песни поют, вот, было такое развлечение в деревне.
[С2: А где это устраивали? У кого-то дома или общее место?]
[КВА:] А по очереди. Каждая девка должна свою очередь соблюдать.
[С1: А это с какого возраста? А это с какого возраста они должны поседки устраивать?]
[КВА:] Ну какие..ну вот какие вот.. уже как начинают загуливать молодёжь, вот так уже да.
[С1: А во сколько начинали в ваше время?]
[КВА:] Ну.. до этого, до армии, тут поседки, они и были, но были старше нас парни, ещё мы там не могли участвовать, только соберёмся посмотреть из заду, в доме там у двери стоим и перенимаем, что старшие братья?, как провожают время. Вот. Потом они уже там начинают всё.. уж там замуж выходят, женятся, туды-сюды, вот, мы замена потом пошли, вот так оно и идёт-идёт-идёт.
[С2: То есть, только по

In [199]:
data = df[['Имя файла','ФИО']].values

In [92]:
from IPython.display import clear_output
import time

In [95]:
data.shape

(495, 2)

In [ ]:
re.findall('(().*?')

In [ ]:
for i in data[100:]:
    print (i)
    array = get_text(i[0])
    informators = ''.join(c for c in i[1] if c.isupper() or c in ',').split(',')
    print(normalize_view(array, informators))
    time.sleep(2)
    clear_output()

In [247]:
for i in range(len(df)):
    old_id = df.iloc[i]['Имя файла']
    print (old_id)
    title = str(df.iloc[i]['Жанр']).replace('"','""')
    informators = ''.join(c for c in df.iloc[i]['ФИО'] if c.isupper() or c in ',').replace(',',', ')
    collector = collectors(df.iloc[i]['ФИ собирателей'])
    city = df.iloc[i]['Деревня']
    address = df.iloc[i]['Место записи (адрес, координаты, etc.)']
    array = get_text(df.iloc[i]['Имя файла'])
    text = normalize_view(array, informators.split(', ')).replace('"','""')
    
    query = 'INSERT INTO Texts (old_id, title, informants, collectors, city, address, raw_text, year)\
    VALUES (\"{}\",\"{}\",\"{}\",\"{}\",\"{}\",\"{}\",\"{}\",{})'.format(
        old_id, title, informators, collector, city, address, text, 2017)
    #print (query)
    print (query)
    cur.execute(query)
    clear_output()
    #print (old_id, '\n', title, '\n', informators, '\n', collector, '\n', city, '\n', address, '\n', text[:200],'\n')

con.commit()

# Block 1 + 2 (finish)

Informators'gender

In [92]:
def g_name(name):
    global m
    res = m.analyze(name)
    fem, mal = 0, 0
    if len(res) == 6:
        res = res[2:]
    for i in res:
        if 'analysis' in i:
            #print (i['analysis'])
            if ',f,' in i['analysis'][0]['gr']:
                fem += 1
            elif ',m,' in i['analysis'][0]['gr']:
                mal += 1
    if fem > 0 and mal == 0:
        return 'f'
    elif fem == 0 and mal > 0:
        return 'm'
    else:
        return None

In [75]:
cur.execute('SELECT id, name FROM Informators')

In [76]:
inf = cur.fetchall()

In [98]:
for i in inf:
    cur.execute('UPDATE Informators  SET gender = ? WHERE id = ?', (g_name(i[1]), i[0]))
    #print (i[1], g_name(i[1]))

In [99]:
con.commit()

In [91]:
m.analyze('Соболькова Зоя Тимофеевна')

[{'analysis': [{'lex': 'собольков',
    'wt': 0.5946199918,
    'qual': 'bastard',
    'gr': 'S,famn,m,anim=(acc,sg|gen,sg)'}],
  'text': 'Соболькова'},
 {'text': ' '},
 {'analysis': [{'lex': 'зоя', 'wt': 1, 'gr': 'S,persn,f,anim=nom,sg'}],
  'text': 'Зоя'},
 {'text': ' '},
 {'analysis': [{'lex': 'тимофеевна', 'wt': 1, 'gr': 'S,patrn,f,anim=nom,sg'}],
  'text': 'Тимофеевна'},
 {'text': '\n'}]

Birthyear(block 1)

In [250]:
cur.execute('SELECT id, bio FROM Informators WHERE old_id < 5000')
data = cur.fetchall()

In [259]:
for i in data:
    date = re.findall('([0-9]{4}) г..?р.',i[1])
    if date != []:
        cur.execute('UPDATE Informators SET birthyear=? WHERE id=?', (date[0], i[0]))
    #print (i[0], '\t', date, '\t', i[1][:20])

In [260]:
con.commit()

Birthplace (block 1)

In [271]:
for i in data:
    place = re.findall('род.? в (.{1,4}? .*?)[.(,]',i[1])
    if place != []:
        cur.execute('UPDATE Informators SET birthplace=? WHERE id=?', (place[0], i[0]))

In [272]:
con.commit()

# New data

In [51]:
def normalize_view(text_array, informants:list):
    inf = {i:i for i in informants}
    copy = informants
    informants = {}
    informants.update({i[-2]:i for i in inf})
    informants.update({i[-2:]:i for i in inf})
    informants.update({i:i for i in inf})
    #if copy == ['ОРА']: informants = {'РА':'ОГА'}
    #print (informants)
    result = []
    for i in text_array:
        if re.match('^[CС0-9, -]{1,7}:.*?$', i):
            result.append('[{}]'.format(i))
            #print ('[{}]'.format(i))
        else:
            r = re.findall('^([А-Яа-яA-Za-z0-9-, ]{1,7}):(.*?$)',i)
            if r:
                    prefix, text = r[0]
                    prefix = ''.join(c for c in prefix if c.isupper() or c.isdigit() or c in ', ')
                    try:
                        result.append('[{}:]{}'.format(informants[prefix], text)) 
                    except KeyError:
                        result.append('[{}:]{}'.format(prefix, text)) 
            else:
                try:
                    result[-1] = result[-1] + '\n' + i
                except:
                    result.append(i)
    result = '\n'.join(result)
    result = result.replace('«','"').replace('»','"')
    return result

In [58]:
data = []
with open('./raw_data/14_11.txt') as f:
    for i in f.read().split('\n\n\n'):
        text = i.strip().split('\n\n')
        a = text[0].split('\n')
        t = re.findall('(.*?):(.*?)-(.*?):(.*?)$',a[1])[0]
        start = int(t[0])*60 + int(t[1])
        end = int(t[2])*60 + int(t[3])
        keywords = re.findall('^.*?:(.*?)$',a[2])[0].strip()
        data.append(tuple(['ЮМК', a[0].strip(), start, end, keywords, normalize_view(text[1].split('\n'), ['CНП'])]))
        #print (a[0].strip(), start, end, keywords)
        #normalize_view(text[1].split('\n'), ['CНП'])
        #print ('\n---------------------------------------------\n'.join(i.strip().split('\n\n')))
        #print ('=================================================================')

In [59]:
data[0]

('ЮМК',
 '-',
 0,
 34,
 '',
 '[CНП:] Тут крылечко, не знаю, тут можно посидеть…\n[С: А там лавочки наверное есть, ну, возле озера.]\n[CНП:] А здесь вот качели, детская площадка.\n[С: Пойдемте.]')

In [63]:
cur.executemany('INSERT INTO Texts (leader, title, start, end, keywords, raw_text)\
VALUES (?, ?, ?, ?, ?, ?)', data)

In [64]:
con.commit()

In [19]:
a = text.split('\n')
a

['Похоронный обряд. Предвестья смерти.',
 '39:15-41:13',
 'Ключевые слова: похоронный обряд, жизнь-смерть, приметы, собака']

In [23]:
re.findall('^(.*?):(.*?)-(.*?):(.*?)$',a[1])

[('39', '15', '41', '13')]

# JSON

In [106]:
from nltk.tokenize import sent_tokenize, TweetTokenizer
tk = TweetTokenizer()
from pymystem3 import Mystem
m = Mystem()
import json

categories = json.load(open('./raw_data/categories.json', 'r', encoding='utf-8'))

def split_sentences(text):
    text = text.replace('[','\n[')
    result = []
    for i in text.split('\n\n'):
        result += sent_tokenize(i)
    return result

def sentence_comment(text):
    c = text.find(']')
    if c > 5:
        return None, m.analyze(text.strip())
    else:
        #print (text[c+1:].strip()+'5')
        return text[:c+1], m.analyze(text[c+1:].strip())
    
def _join_text(beginning, sentence):
        
        #meta
    
        if beginning is not None and beginning != '':
            text = beginning+' '
        else:
            text = ''
        for key, word in enumerate(sentence['words']):
            if 'wf' not in word:
                continue
            sentence['words'][key]['off_start'] = len(text)
            if word['wtype'] == 'word':
                text += word['wf'] + ' '
                sentence['words'][key]['off_end'] = len(text) - 1
            else:
                if word['wf'].startswith(('(', '[', '{', '<', '“')):
                    text += word['wf']
                    sentence['words'][key]['off_end'] = len(text)
                elif word['wf'].startswith((')', ']', '}', '>', '.', ':',',', '?', '!', '”', '…')):
                    if text.endswith(' '):
                        sentence['words'][key]['off_start'] -= 1
                        text = text[:-1]
                    text += word['wf'] + ' '
                    sentence['words'][key]['off_end'] = len(text) - 1
                else:
                    text += word['wf'] + ' '
                    sentence['words'][key]['off_end'] = len(text) - 1
            sentence['words'][key]['sentence_index'] = key
            sentence['words'][key]['next_word'] = key+1
        sentence['text'] = text
        if beginning is not None:
            sentence['words'] = [{'wf':beginning, 'wtype':'comment', 
                                  'off_start':0, 'off_end':len(beginning)}]+sentence['words']
        return sentence
     
def mystem_interpreter(word, language='russian'):
    result = []
    if 'analysis' in word:
    #if True:
        for i in word['analysis']:
            lex = i['lex']
            variants = i['gr'].split('=')
            variants[0] = variants[0].split(',')
            variants[1] = [x.split(',') for x in variants[1].strip('()').split('|')]
            if variants[1] == [['']]: 
                variants[1] = []
                cur = {'lex':lex}
                for var in variants[0]:
                    cur['gr.{}'.format(categories[language][var])] = var
                result.append(cur)
            else:
                for j in variants[1]:
                    cur = {'lex':lex}
                    for var in variants[0]+j:
                        cur['gr.{}'.format(categories[language][var])] = var
                    result.append(cur)
        #print (result)
        return {
            'wtype':'word',
            'wf':word['text'],
            'ana': result
        }
    else:
        return {
            'wtype':'punkt',
            'wf':word['text']
        }
    
def sentences(text):
    text = split_sentences(text)
    result = []
    for i in text:
        sentence = sentence_comment(i)
        #print (sentence)
        #print (sentence)
        cur = []
        for j in sentence[1]:
            cur.append(mystem_interpreter(j))
        result.append(_join_text(sentence[0], {'words':cur}))
    return result

In [93]:
print(data[4][-1])

[CНП:] … Танцевали мы, пели частушки про Масленицу.
[С: А вы не напоете сейчас какие-нибудь частушки? Может, вспомните?]
[CНП:] 
Как на Масляной неделе
Санки с горочки летели!
Дунька с горки катится,
На ней бородово платьице! 
<смеется>
[С: Здорово!]
[CНП:] Вот это на Масленицу мы пели.<смеется>


In [107]:
for i in sentences(data[4][-1]):
    print ('|{}|'.format(i['text']))

|[CНП:] …  Танцевали   мы,  пели   частушки   про   Масленицу. 
 |
|[С:  А   вы   не   напоете   сейчас   какие-нибудь   частушки? 
 |
|Может,  вспомните?]
 |
|[CНП:] Как   на   Масляной   неделе 
 Санки   с   горочки   летели! 
 |
|Дунька   с   горки   катится,
 На   ней   бородово   платьице! 
 |
|<смеется>
 |
|[С:  Здорово!]
 |
|[CНП:] Вот   это   на   Масленицу   мы   пели. <смеется>
 |


In [108]:
sentences(data[4][-1])

[{'words': [{'wtype': 'punkt',
    'wf': '[',
    'off_start': 0,
    'off_end': 1,
    'sentence_index': 0,
    'next_word': 1},
   {'wtype': 'word',
    'wf': 'CНП',
    'ana': [],
    'off_start': 1,
    'off_end': 4,
    'sentence_index': 1,
    'next_word': 2},
   {'wtype': 'punkt',
    'wf': ':] ',
    'off_start': 4,
    'off_end': 7,
    'sentence_index': 2,
    'next_word': 3},
   {'wtype': 'punkt',
    'wf': '… ',
    'off_start': 7,
    'off_end': 9,
    'sentence_index': 3,
    'next_word': 4},
   {'wtype': 'word',
    'wf': 'Танцевали',
    'ana': [{'lex': 'танцевать',
      'gr.pos': 'V',
      'gr.asp': 'ipf',
      'gr.trans': 'tran',
      'gr.tense': 'praet',
      'gr.num': 'pl',
      'gr.mood': 'indic'}],
    'off_start': 10,
    'off_end': 19,
    'sentence_index': 4,
    'next_word': 5},
   {'wtype': 'punkt',
    'wf': ' ',
    'off_start': 20,
    'off_end': 21,
    'sentence_index': 5,
    'next_word': 6},
   {'wtype': 'word',
    'wf': 'мы',
    'ana': [{'lex'

In [88]:
print (data[4][-1])

[CНП:] … Танцевали мы, пели частушки про Масленицу.
[С: А вы не напоете сейчас какие-нибудь частушки? Может, вспомните?]
[CНП:] 
Как на Масляной неделе
Санки с горочки летели!
Дунька с горки катится,
На ней бородово платьице! 
<смеется>
[С: Здорово!]
[CНП:] Вот это на Масленицу мы пели.<смеется>


In [89]:
split_sentences(data[4][-1])

['\n[CНП:] … Танцевали мы, пели частушки про Масленицу.',
 '[С: А вы не напоете сейчас какие-нибудь частушки?',
 'Может, вспомните?]',
 '[CНП:] \nКак на Масляной неделе\nСанки с горочки летели!',
 'Дунька с горки катится,\nНа ней бородово платьице!',
 '<смеется>',
 '[С: Здорово!]',
 '[CНП:] Вот это на Масленицу мы пели.<смеется>']

In [ ]:
"meta": {
    "speaker": "AP",
    "gender": "M",
    "year": "2017"
  }